<h1>Intuitive Guide to Gaussian Processes</h1>

<br>

Gaussian processes have many uses that range robotics, machine learning tasks, or simple fitting of function to data points (regression). The importance of GP cannot be understated, but <i>INTUTITVELY</i> understanding GP's is easier said than done. When conducting research I found limited resources that were difficult to understand. In this notebook I hope to reduce the amount of complexity and provide and easy way to understand GP's and illustrate why they are very important. 


Without futher ado, lets get started:


Before we start here are a few questions with answer that we must know...

<br>
<br>
        
1) What is a Gaussian Process?  
        
        - Gaussian processes are a powerful tool that allow us to make predictions about our data by incorporating prior knowledge.


2) Are GP's regression problems? What is regression?  

      - Regression is the process of receiving points (training data) and creating a function/prediction from those points. There can be multiple possible functions that fit our data, so we need a way to track how CERTAIN/UNCERTAIN these functions/predictions will be.
      
      
      
                        Below you can see that multiple functions work with the observed points.
                                Click the "add random drawing" and a function will appear. 
<br>
<br>


In [1]:
from utils.hide_toggle import hide_toggle
hide_toggle(for_next = True)

ModuleNotFoundError: No module named 'utils'

In [ ]:
'''
    Run block to display of adding functions to sample points
'''


from GPR import GPR
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interact, widgets
from plots import *
from GPR import GPR
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interact, widgets
from plots import *

x_values = np.array([0, 0.3, 1, 3.1, 4.7])
y_values = np.array([1, 0, 1.4, 0, -0.9])
x = np.arange(-1,7,0.1)

model = GPR(x_values, y_values)

mean = model.predict(x)
covariance_matrix = model._memory['covariance_matrix']

fig1 = go.FigureWidget(data=[dot_scatter(x_dots=x_values, y_dots=y_values)])
fig1 = update_layout_of_graph(fig1, title='Random Regression functions of the Gaussian process')

button = widgets.Button(description='Add random drawing')


def update(_):
    with fig1.batch_update():
        fig1.add_trace(
            line_scatter(x_lines=x,
                         y_lines=np.random.multivariate_normal(
                             mean, covariance_matrix),
                         name_line='random function',
                         showlegend=False))
        fig1.add_trace(
            dot_scatter(x_dots=x_values, y_dots=y_values, showlegend=False))
        
        fig1.show()

button.on_click(update)
widgets.VBox([fig1, button])



3) What is a Gaussian Distribution? 
        
       - A GP is defined the equation below and looks very similar to a bell graph. For each points or value assigned will produce a gaussian distribution and each point encompasses a mean and a covariance. mean describing the expected value of the function, whereas the covariance matrix defines the shape of the uncertainty. 
      

<div>
    <br>
    <td>
        <img src="https://www.gstatic.com/education/formulas2/443397389/en/normal_distribution.svg" width="400"/> [2]
    </td>
    <br>
</div>

                        This equation can be specified using mean vector and covariance matrix. 


<br>
<br>


4) Why is does being certain/uncertain matter?  
      
      - As seen above uncertainy determines how far away a function can deviate from the center (mean). By measuring uncertainty we can be more certain. Kinda funny saying it out loud but it applies in this situation and is in fact an effective learning process for exploring and a better understanding our search space.
      
                      Below we illustrate uncertainy changes as you add more sample points. 
      

In [ ]:
hide_toggle(for_next = True)

In [ ]:
'''

    Sometimes you will have to uncomment the fig.show() in the update section to see graph. 

'''


def plot_GPR(data_x, data_y, model, x, visible=True) -> list:
    
    mean = model.predict(x)
    std = np.sqrt(model._memory['variance'])
    
    
    data = [
        
        uncertainty_area_scatter(x_lines=x, y_lower=mean - 1 * std, y_upper=mean + 1 * std,
                name=f"uncertainty",
                visible=visible),

        line_scatter(x_lines=x, y_lines=mean, visible=visible),
        dot_scatter(x_dots=data_x, y_dots=data_y, visible=visible)
    
    ]
    
    return data

model = GPR(x_values[:1], y_values[:1])

data = plot_GPR(data_x=x_values[:1], data_y=y_values[:1], x=x, model=model)

fig2 = go.FigureWidget(data=data)
fig2 = update_layout_of_graph(fig2, title='Prediction (i.e. mean) of GPR')

button = widgets.Button(description='Add data point')

number_of_points = 1

# we update the y values of our 4 scatter plots 
def update(_):
    with fig2.batch_update():
        global number_of_points
        if number_of_points < len(x_values):
            number_of_points += 1
            model = GPR(x_values[:number_of_points],
                        y_values[:number_of_points])
            mean = model.predict(x)
            fig2.data[1].y = mean
            y_upper = mean + 1 * np.sqrt(model._memory['variance'])
            y_lower = mean - 1 * np.sqrt(model._memory['variance'])
            fig2.data[0].y = np.concatenate((y_upper, y_lower[::-1]))
            fig2.data[2].x = x_values[:number_of_points]
            fig2.data[2].y = y_values[:number_of_points]
            fig2.show() # uncomment if graph shows

button.on_click(update)
widgets.VBox([fig2, button])

We have discussed the basics of guassian processes and this will be vital to understanding Deep Gaussian Processes. There are many different layers including Marginalization and Conditioning, which can be found in these helpful sources below.



https://distill.pub/2019/visual-exploration-gaussian-processes/
https://www.ritchievink.com/blog/2019/02/01/an-intuitive-introduction-to-gaussian-processes/
